# 🐍 Snake Q-Learning Tutorial

This notebook demonstrates how to train a reinforcement learning agent to play Snake using Q-Learning.

## What You'll Learn
- How the Snake game environment works
- Q-Learning fundamentals
- Training and evaluating an RL agent
- Visualizing learning progress

## 1. Setup and Imports

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src directory to path
sys.path.insert(0, '../src')

from Snake import SnakeGame, Snake
from QLearningAgent import QLearningAgent
from config import PresetConfigs
from utils import plot_training_curves, analyze_state_coverage, print_training_summary

print("✅ All imports successful!")

## 2. Explore the Environment

Let's create a game and see how the state representation works.

In [ ]:
# Create a small game
game = SnakeGame(width=8, height=8)

print("Initial Game State:")
game.display()

print(f"\nSnake length: {game.length}")
print(f"State vector: {game.state}")
print(f"State number: {game._get_state_number()}")

### Understanding the State Vector

The state is an 8-element binary vector:
- **Bits 0-3**: Obstacles in each direction (↑→↓←)
- **Bits 4-7**: Food direction (↑→↓←)

Let's make a few moves and see how the state changes:

In [ ]:
# Make a random move
import random

action = random.randint(0, 3)
direction_names = ["UP", "RIGHT", "DOWN", "LEFT"]

print(f"Taking action: {direction_names[action]}")
state, reward, game_over, score = game.step(action)

game.display()
print(f"\nReward: {reward}")
print(f"New state: {game.state}")
print(f"Score: {score}")

## 3. Create and Configure an Agent

Now let's create a Q-Learning agent with custom hyperparameters.

In [ ]:
# Create agent with default parameters
agent = QLearningAgent(
    gamma=0.8,          # Discount factor - how much to value future rewards
    epsilon=0.2,        # Exploration rate - 20% random actions during training
    learning_rate=0.9   # Learning rate for Q-value updates
)

print("Agent created!")
print(f"Q-table shape: {agent.q_table.shape}")
print(f"Number of states: {agent.num_states}")
print(f"Number of actions: {agent.num_actions}")

## 4. Train the Agent

Let's train the agent for 5000 episodes. This should take a few minutes.

**Note**: Set `num_episodes` to 1000 for a quicker demo.

In [ ]:
# Training configuration
BOARD_SIZE = 16
NUM_EPISODES = 5000  # Change to 1000 for quick demo
EVAL_FREQUENCY = 100

print(f"Starting training for {NUM_EPISODES} episodes...\n")

training_results = agent.train(
    board_size=BOARD_SIZE,
    num_episodes=NUM_EPISODES,
    eval_frequency=EVAL_FREQUENCY,
    eval_episodes=25,
    save_q_history=False,  # Set to True to save convergence data
    verbose=True
)

print("\n✅ Training complete!")

## 5. Analyze Training Results

Let's visualize the learning progress.

In [ ]:
# Plot training progress
agent.plot_training_progress(save_path='../results/notebook_training_progress.png')

In [ ]:
# Analyze state space coverage
coverage = analyze_state_coverage(agent.q_table)

print("State Space Analysis:")
print(f"Total possible states: {coverage['total_states']}")
print(f"States visited: {coverage['visited_states']} ({coverage['visit_percentage']:.1f}%)")
print(f"Fully explored states: {coverage['fully_explored_states']} ({coverage['full_exploration_percentage']:.1f}%)")

## 6. Evaluate the Trained Agent

Let's test how well our agent performs!

In [ ]:
# Evaluate over 100 games
avg_score, scores = agent.evaluate(board_size=BOARD_SIZE, num_episodes=100)

print("Evaluation Results (100 games):")
print(f"Average score: {avg_score:.2f}")
print(f"Best game: {max(scores)}")
print(f"Worst game: {min(scores)}")
print(f"Standard deviation: {np.std(scores):.2f}")
print(f"\nGames with score ≥10: {sum(1 for s in scores if s >= 10)}/100")
print(f"Games with score ≥20: {sum(1 for s in scores if s >= 20)}/100")

In [ ]:
# Visualize score distribution
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(scores, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(avg_score, color='r', linestyle='--', linewidth=2, label=f'Mean: {avg_score:.1f}')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title('Score Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(scores, 'b-', alpha=0.6)
plt.axhline(avg_score, color='r', linestyle='--', label=f'Mean: {avg_score:.1f}')
plt.xlabel('Game Number')
plt.ylabel('Score')
plt.title('Score per Game')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/notebook_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Watch the Agent Play

Let's watch one game in action (text-based visualization).

In [ ]:
# Play one game and show each step
game = SnakeGame(BOARD_SIZE, BOARD_SIZE)
state = game._get_state_number()
game_over = False
steps = 0
max_steps = 200

print("Initial State:")
game.display()
print(f"Score: {game.length}\n")

move_history = []

while not game_over and steps < max_steps:
    action = agent.select_action(state, training=False)
    state, reward, game_over, score = game.step(action)
    
    move_history.append({
        'step': steps,
        'action': ['UP', 'RIGHT', 'DOWN', 'LEFT'][action],
        'reward': reward,
        'score': score
    })
    
    steps += 1

print("Final State:")
game.display()
print(f"\nGame Over! Final Score: {score}")
print(f"Total steps: {steps}")

# Show last 10 moves
print("\nLast 10 moves:")
for move in move_history[-10:]:
    print(f"  Step {move['step']}: {move['action']} -> Reward: {move['reward']:+.0f}, Score: {move['score']}")

## 8. Save and Load the Model

You can save your trained agent and load it later.

In [ ]:
# Save the model
agent.save('../models/notebook_trained_agent.pkl')

print("Model saved!")

In [ ]:
# Load the model
new_agent = QLearningAgent()
new_agent.load('../models/notebook_trained_agent.pkl')

# Test loaded agent
avg, _ = new_agent.evaluate(board_size=BOARD_SIZE, num_episodes=10)
print(f"Loaded agent average score: {avg:.2f}")

## 9. Experiment with Hyperparameters

Try different hyperparameter combinations and see how they affect learning!

In [ ]:
# Quick comparison of different gamma values
gamma_values = [0.6, 0.8, 0.9]
results_comparison = {}

for gamma in gamma_values:
    print(f"\nTraining with gamma={gamma}...")
    test_agent = QLearningAgent(gamma=gamma, epsilon=0.2)
    test_agent.train(board_size=8, num_episodes=1000, verbose=False)
    avg, _ = test_agent.evaluate(board_size=8, num_episodes=25)
    results_comparison[gamma] = avg
    print(f"  Average score: {avg:.2f}")

# Plot comparison
plt.figure(figsize=(8, 5))
plt.bar([str(g) for g in gamma_values], list(results_comparison.values()), alpha=0.7, edgecolor='black')
plt.xlabel('Gamma (γ)')
plt.ylabel('Average Score')
plt.title('Impact of Discount Factor on Performance')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"\nBest gamma: {max(results_comparison, key=results_comparison.get)}")

## 10. Summary and Next Steps

### What We Learned:
- ✅ How the Snake environment encodes states
- ✅ Q-Learning algorithm basics
- ✅ Training and evaluating an RL agent
- ✅ Analyzing learning progress
- ✅ Saving and loading models

### Try These Challenges:
1. **Increase board size**: Train on a 20x20 board
2. **Modify rewards**: Change reward values in `Snake.py`
3. **Different exploration**: Try epsilon=0.3 or 0.1
4. **Longer training**: Train for 20,000+ episodes
5. **Compare agents**: Train multiple agents and compare performance

### Advanced Topics:
- Implement Deep Q-Networks (DQN) for larger state spaces
- Add more features to the state representation
- Try different RL algorithms (SARSA, Double Q-Learning)
- Create a visual animation of the agent playing